In [1]:
!wget http://cb.lk/covid_19


!unzip covid_19

--2020-05-22 06:42:51--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 159.89.170.74
Connecting to cb.lk (cb.lk)|159.89.170.74|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2020-05-22 06:42:51--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|159.89.170.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2020-05-22 06:42:53--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2020-05-22 06:42:53--  https://www.dropbox.com/s/raw/7rjw

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

# CNN model in keras
# 32 feature extractors initially 
# 224, 224, 3 means height, width, no of channels
# Xray images look b&w, but are RGB, therefore we need 3 channels
# (3,3) feature map size/kernal size/filter size is a default size

model = Sequential()
model.add(Conv2D(32, kernel_size = (3,3), activation = "relu", input_shape = (224, 224, 3)))
model.add(Conv2D(64, (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

# adding dropout to check for overfitting
model.add(Dropout(0.25))

# another conv layer
model.add(Conv2D(64, (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))


# another conv layer 
model.add(Conv2D(128, (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation = "sigmoid"))

model.compile(loss = keras.losses.binary_crossentropy, optimizer = "adam", metrics = ['accuracy'])


Using TensorFlow backend.


In [0]:
mode.summary()

NameError: ignored

In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 54, 54, 64)       

In [4]:
# Train from scratch

# Image augmentation 
# Training images augmentation
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

# Test images augmentation
test_dataset = image.ImageDataGenerator(
    rescale = 1./255
)

train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 224 images belonging to 2 classes.


In [5]:
# to get the classes of the outputs
# 0-> Covid
# 1 -> Normal
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [6]:
validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 60 images belonging to 2 classes.


In [8]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = 8,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps = 2
)

Epoch 1/10
8/8 [==============================] - 86s 11s/step - loss: 0.1783 - accuracy: 0.9531 - val_loss: 0.1102 - val_accuracy: 0.9833
Epoch 2/10
8/8 [==============================] - 78s 10s/step - loss: 0.1002 - accuracy: 0.9688 - val_loss: 0.0695 - val_accuracy: 0.9833
Epoch 3/10
8/8 [==============================] - 78s 10s/step - loss: 0.0764 - accuracy: 0.9648 - val_loss: 0.0942 - val_accuracy: 0.9833
Epoch 4/10
8/8 [==============================] - 81s 10s/step - loss: 0.1201 - accuracy: 0.9531 - val_loss: 0.0655 - val_accuracy: 1.0000
Epoch 5/10
8/8 [==============================] - 78s 10s/step - loss: 0.0804 - accuracy: 0.9688 - val_loss: 0.0108 - val_accuracy: 0.9667
Epoch 6/10
8/8 [==============================] - 78s 10s/step - loss: 0.0785 - accuracy: 0.9727 - val_loss: 0.2025 - val_accuracy: 0.9667
Epoch 7/10
8/8 [==============================] - 83s 10s/step - loss: 0.0936 - accuracy: 0.9727 - val_loss: 0.0151 - val_accuracy: 0.9667
Epoch 8/10
8/8 [===========

In [0]:
# Testing our model on 1 image to make prediction

test_image = image.load_img('testing_image.jpg')
test_image = image.img_to_array(test_image)
test_image.resize(224, 224, 3)
test_image = np.expand_dims(test_image, axis = 0)
res = model.predict(test_image)


In [10]:
print(res)

[[0.]]


In [11]:
if res[0][0] == 1:
  prediction = 'Normal'
else:
  prediction = "Corona Positive"

print("The patient is", prediction)

The patient is Corona Positive
